 # Predict at Scale



 This script:

 - Loads the full processed Sentiment140 dataset

 - Randomly samples 100 tweets for prediction

 - Encodes these 100 examples in batches using the all-mpnet-base-v2 sentence transformer

 - Loads the final trained classifier model

 - Predicts probabilities for each example

 - Applies a threshold (0.5) to classify as 'sarcastic' or 'literal'

 - Shows class distribution and random sample of predictions

 - (Optionally) saves predictions to CSV

 - Uses the GPT API to classify each tweet in the sample

 - Compares model predictions to LLM labels and reports accuracy, TP, TN, FP, FN rates

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from keras.models import load_model
import openai
import json
import time
from google.colab import userdata # Import userdata to access secrets


In [3]:
# Load the full dataset
full_df = pd.read_csv('processed_sentiment140.csv')

# Select a random sample of 100 examples for prediction
predict_df = full_df.sample(n=100, random_state=42).copy()


In [4]:
# Load the same sentence transformer model used in training
ST_model = SentenceTransformer('all-mpnet-base-v2')


In [5]:
# Encode the examples in batches
batch_size = 32
all_embeddings = []
print('Encoding 100 examples in batches...')
for i in tqdm(range(0, len(predict_df), batch_size)):
    batch = predict_df['cleaned_text'].iloc[i:i+batch_size].tolist()
    batch_embed = ST_model.encode(batch)
    all_embeddings.extend(batch_embed)
all_embeddings = np.array(all_embeddings)


Encoding 100 examples in batches...


100%|██████████| 4/4 [00:00<00:00,  4.26it/s]


In [6]:
# Load the final trained model
final_model = load_model('final_classifier.h5')


In [7]:
# Predict probabilities
probs = final_model.predict(all_embeddings)

# Apply threshold to classify
threshold = 0.5
predict_df['predicted_probability'] = probs.flatten()
predict_df['predicted_label'] = (probs.flatten() > threshold).astype(int)

# Map back to string labels if needed (optional)
label_map = {1: 'sarcastic', 0: 'literal'}
predict_df['predicted_label_str'] = predict_df['predicted_label'].map(label_map)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


In [8]:
# Show a sample of predictions
print(predict_df[['text', 'predicted_probability', 'predicted_label', 'predicted_label_str']].head())

# Show statistics of predicted classes
print("\nPredicted class distribution:")
print(predict_df['predicted_label_str'].value_counts())


                                                     text  \
725031  Was sending my last tweet typed in moon and it...   
352959  In Tha House, Sore From Working Out...Kinda Mi...   
601371  @Queluver55 just wanted to say hi and that her...   
425948  TGIF but then its TGIS then oh man its Sunday....   
567048  in time you will get it young grass hopper@nat...   

        predicted_probability  predicted_label predicted_label_str  
725031               0.431566                0             literal  
352959               0.034415                0             literal  
601371               0.001978                0             literal  
425948               0.804007                1           sarcastic  
567048               0.707501                1           sarcastic  

Predicted class distribution:
predicted_label_str
literal      73
sarcastic    27
Name: count, dtype: int64


In [9]:
# Optionally, save predictions to a new CSV
predict_df.to_csv('predictions_100.csv', index=False)
print('Saved predictions for 100 examples to predictions_100.csv')


Saved predictions for 100 examples to predictions_100.csv


In [10]:
# Load OpenAI API key from Colab secrets
openai.api_key = userdata.get('OpenAI_API_Key')

# Initialize OpenAI client
client = openai.OpenAI(api_key=openai.api_key)

def classify_with_llm(text, max_retries=3, wait_time=0.5):
    prompt = (
        "Classify the following tweet as either 'literal' or 'sarcastic'. "
        "Respond with only one word: literal or sarcastic.\n"
        f"Tweet: {text}\nLabel:"
    )
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4.1-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that classifies tweets as literal or sarcastic. Respond with only one word: literal or sarcastic."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
                max_tokens=1
            )
            label = response.choices[0].message.content.strip().lower()
            if 'lit' in label:
                return 'literal'
            elif 'sar' in label:
                return 'sarcastic'
            else:
                return label
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Attempt {attempt + 1} failed: {str(e)}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
                wait_time *= 2
            else:
                print(f"All {max_retries} attempts failed. Last error: {str(e)}")
                return None


In [11]:
# Apply LLM classification to the 100 samples (this will use tokens and take time)
llm_labels = []
for text in tqdm(predict_df['text'], desc='Classifying with LLM'):
    llm_labels.append(classify_with_llm(text))
predict_df['llm_label'] = llm_labels


Classifying with LLM: 100%|██████████| 100/100 [00:46<00:00,  2.17it/s]


In [12]:
# Compare model predictions with LLM labels
def get_tp_tn(row):
    if row['predicted_label_str'] == 'sarcastic' and row['llm_label'] == 'sarcastic':
        return 'TP'
    elif row['predicted_label_str'] == 'literal' and row['llm_label'] == 'literal':
        return 'TN'
    elif row['predicted_label_str'] == 'sarcastic' and row['llm_label'] == 'literal':
        return 'FP'
    elif row['predicted_label_str'] == 'literal' and row['llm_label'] == 'sarcastic':
        return 'FN'
    else:
        return 'Other'

predict_df['llm_agreement'] = predict_df.apply(get_tp_tn, axis=1)


In [13]:
# Report accuracy and confusion matrix
accuracy = (predict_df['predicted_label_str'] == predict_df['llm_label']).mean()
tp = (predict_df['llm_agreement'] == 'TP').sum()
tn = (predict_df['llm_agreement'] == 'TN').sum()
fp = (predict_df['llm_agreement'] == 'FP').sum()
fn = (predict_df['llm_agreement'] == 'FN').sum()
print(f'LLM agreement accuracy: {accuracy:.3f}')
print(f'TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}')


LLM agreement accuracy: 0.700
TP: 16, TN: 54, FP: 11, FN: 19
